In [1]:
from qdrant_client import QdrantClient


# Initialize the client
client = QdrantClient(":memory:")


In [2]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('styles.csv', on_bad_lines='skip')

# Extract the 'productDisplayName' column into a list
product_display_names = df['productDisplayName'].astype(str).tolist()


In [3]:
# Function to split list into batches of specified size
def batch_list(lst, batch_size):
    for i in range(0, len(lst), batch_size):
        yield lst[i:i + batch_size]

# Define batch size
batch_size = 1000

# Process the batches
for idx, batch in enumerate(batch_list(product_display_names, batch_size)):
    start_id = idx * batch_size + 1
    end_id = start_id + len(batch) - 1
    doc_ids = list(range(start_id, end_id + 1))
    client.add(
        collection_name="product_list",
        documents=batch,
        ids=doc_ids,
    )

In [4]:
from dspy.retrieve.qdrant_rm import QdrantRM
import dspy


qdrant_retriever_model = QdrantRM("product_list", client, k=3)


ollama_model = dspy.OllamaLocal(model="llama3",model_type='text',
                                max_tokens=350,
                                temperature=0.1,
                                top_p=0.8, frequency_penalty=1.17, top_k=40)


dspy.settings.configure(lm= ollama_model, rm=qdrant_retriever_model)

In [5]:
class Items(dspy.Signature):
    """
    List of extracted clothing items and accessories
    """
    prompt = dspy.InputField()
    terms = dspy.OutputField(format=list)


class FindItems(dspy.Module):
    """
    Extract clothing items and accessories from a user query
    """
    def __init__(self):
        super().__init__()
        self.item_extractor = dspy.Predict(Items)

    def forward(self, question):
        instruction = (
            "Return a list of clothing items and accesores to be worn by a user, given their preferences"
            f"Preferences: {question}"
        )
        prediction = self.item_extractor(
            prompt=instruction
        )
        return prediction.terms

In [6]:
module = FindItems()

In [7]:
response = module("Tripy party with clowns and lot if music in New York")

In [8]:
print(response)

Here is the extracted list of clothing items and accessories:

**List**

* Sequined top (for a flashy outfit)
* Glittery pants (to match the sparkly atmosphere)
* Clown-inspired shoes (with bright colors and oversized bows)
* Statement necklace with flashing lights or glittering beads
* Oversized sunglasses with colorful frames
* A funky hat shaped like a clown's nose or an animal ear

These items are inspired by the user's preference for a trippy party with clowns and lots of music in New York, aiming to create a playful, whimsical outfit that matches the energetic vibe.


In [9]:
trainset_dict= {
    "desert_wear": "* Lightweight, breathable long-sleeve shirt\n* Wide-brimmed hat\n* Sunglasses with UV protection\n* Cargo pants\n* Lightweight scarf or shemagh\n* Hiking boots with good ventilation\n* Moisture-wicking socks\n* Sunscreen\n* Hydration pack or water bottle\n* Lightweight jacket for cooler evenings",
    "caribbean_party": "* Flowy sundress with tropical print\n* Beach hat with colorful feathers\n* Statement necklace with seashells and beads\n* Flip flops in bright colors\n* Sunglasses with brightly colored frames\n* Lightweight, short-sleeve shirt with floral patterns\n* Comfortable shorts\n* Colorful bangles or bracelets\n* Straw tote bag\n* Lightweight scarf or wrap",
    "award_function": "* Elegant evening gown or tuxedo\n* Statement jewelry (earrings, necklace, bracelet)\n* Dress shoes or heels\n* Clutch or small evening bag\n* Cufflinks and tie (for tuxedo)\n* Evening makeup and hairstyle\n* Silk scarf or shawl\n* Formal watch\n* Perfume or cologne\n* Pocket square (for tuxedo)"
}

In [10]:
trainset = [dspy.Example(question=key, answer=value) for key, value in trainset_dict.items()]

In [11]:
from dspy import teleprompt
teleprompter = teleprompt.LabeledFewShot()
optimized_Find_Items = teleprompter.compile(student=FindItems(), trainset=trainset)

In [12]:
module = optimized_Find_Items("Belly Dancers with fire of Arabia")

In [13]:
module

'Here is the extracted list of clothing items and accessories:\n\n**Belly Dancers with Fire of Arabia**\n\n* Harem pants (flowing, loose-fitting pants)\n* Midriff-baring tops (revealing the belly button)\n* Belly chain or belt (adorned with coins or other decorative elements)\n* Headscarf or hijab (covering the hair and neck)\n* Arm warmers or gauntlets (protective sleeves for fire dancing)\n* Fire-resistant gloves\n* Sparkly, beaded jewelry (necklaces, earrings, etc.)\n* High-heeled ankle boots or sandals with intricate designs\n* Flowing cape or shawl to add drama while performing\n\nThese items are inspired by traditional Middle Eastern and North African clothing styles, adapted for the unique demands of belly dancing with fire.'

In [15]:
def parse_clothing_items(text):
    # Split the text into lines
    lines = text.split('\n')

    # Initialize an empty list to hold the items
    items = []

    # Loop through each line
    for line in lines:
        # Check if the line starts with '*'
        if line.strip().startswith('* '):
            # Remove the '*' and any leading/trailing whitespace, then add to the list
            item = line.strip()[2:].strip()
            items.append(item)

    return items


In [ ]:
class Style(dspy.Signature):
    """
    Generate styling advise, given a list of products
    """
    product_list = dspy.InputField(format=list)
    answer = dspy.OutputField()

In [ ]:
class Styling_Advise(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_response = dspy.ChainOfThought(Style, n=3)
        self.retrieve = dspy.Retrieve(k=6)

    def forward(self, question):
        item_list= parse_clothing_items(optimized_Find_Items(question))
        product_list = []
        for item in item_list:
            product_list += self.retrieve(item).passages

        response = self.generate_response(product_list= product_list)
        return response.answer